In [ ]:
import pandas as pd
#loading dataset
df=pd.read_csv("dfs.csv")
df.head()
df.info()
# check missing values
print(df.isnull().sum())
df.describe()

Outlier_Detection_Z_Score

In [ ]:
import numpy as np
def detect_outliers_zscore(data,threshold=3):
    z_scores=np.abs((data-data.mean())/data.std())
    return z_scores>threshold
#Applying to numeric columns z_scores
numeric_cols=df.select_dtypes(include=[np.number]).columns
outliers_zscore=df[numeric_cols].apply(detect_outliers_zscore)
#Displaying outliers in each column
outliers_zscore.sum()

IQR [Q1-1.5*IQR,Q3+1.5*IQR]

In [ ]:
def detect_outliers_iqr(data):
    Q1=data.quantile(0.25)
    Q3=data.quantile(0.75)
    IQR=Q3-Q1
    lower_bound=Q1-1.5*IQR
    upper_bound=Q3+1.5*IQR
    return (data<lower_bound)|(data>upper_bound)
#Applying IQR to numeric data
outliers_iqr=df[numeric_cols].apply(detect_outliers_iqr)
#Show the number of outliers in each column
print(outliers_iqr.sum())
                                            

combine z_score && IQR

In [ ]:
outliers_combined=(outliers_zscore & outliers_iqr)
outliers_combined.sum()

Remove outliers

In [ ]:
df_cleaned=df[~outliers_combined.any(axis=1)]      
print(f"number of columns after remove outliers {df_cleaned.shape[0]}")

Replacing outliers with either the mean or median

In [ ]:
df_no_outliers=df.copy()
for col in numeric_cols:
    median=df[col].median()
    df_no_outliers.loc[outliers_combined[col],col]=median

Normalization

In [ ]:
from sklearn.preprocessing import MinMaxScaler
#normalize data 
scaler=MinMaxScaler()
df_normalized=df_cleaned.copy()
df_normalized[numeric_cols]=scaler.fit_transform(df_cleaned[numeric_cols])
print("normalized data: ")
df_normalized.head()

In [ ]:
df_normalized.to_csv("outliers_detection.csv",index=False)
print("number of outliers with z-score : ",outliers_zscore.sum())
print("number of outliers with iqr :" ,outliers_iqr.sum())
print("number of outliers with combined :",outliers_combined.sum())

illustration

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns
plt.figure(figsize=(10,5))
plt.subplot(1,2,1)
sns.boxplot(y=df["Porosity"])
plt.title("before removing outliers ")
plt.subplot(1,2,2)
sns.boxplot(y=df_cleaned['Porosity'])
plt.title("after removing outliers")
plt.show()